In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0"  # Set the GPU 2 to use
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')
# tf.debugging.set_log_device_placement(False)

2024-08-11 10:34:46.823217: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-11 10:34:47.114460: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 10:34:47.114488: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 10:34:47.115587: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-11 10:34:47.230042: I tensorflow/core/platform/cpu_feature_g

Num GPUs Available:  1


2024-08-11 10:34:50.307997: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-11 10:34:50.434898: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-11 10:34:50.435120: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import numpy as np

In [3]:
n_class = 3

In [5]:
from keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Reshape
from keras.models import Model

input = Input(shape=(448,448,3))
c1 = Conv2D(64,(7,7),(2,2),padding='same',activation='relu')(input)
m1 = MaxPool2D((2, 2), strides=(2,2))(c1)
c2 = Conv2D(192,(3,3),padding='same',activation='relu')(m1)
m2 = MaxPool2D((2, 2), strides=(2,2))(c2)
c3_1 = Conv2D(128, (1,1), activation='relu')(m2)
c3_2 = Conv2D(256, (3,3), padding='same',activation='relu')(c3_1)
c3_3 = Conv2D(256, (1,1), activation='relu')(c3_2)
c3_4 = Conv2D(512, (3,3), padding='same',activation='relu')(c3_3)
m3 = MaxPool2D((2, 2), strides=(2,2))(c3_4)
c4_1 = Conv2D(256, (1,1), padding='same',activation='relu')(m3)
c4_2 = Conv2D(512, (3,3), padding='same',activation='relu')(c4_1)
c4_3 = Conv2D(256, (1,1), padding='same',activation='relu')(c4_2)
c4_4 = Conv2D(512, (3,3), padding='same',activation='relu')(c4_3)
c4_5 = Conv2D(256, (1,1), padding='same',activation='relu')(c4_4)
c4_6 = Conv2D(512, (3,3), padding='same',activation='relu')(c4_5)
c4_7 = Conv2D(256, (1,1), padding='same',activation='relu')(c4_6)
c4_8 = Conv2D(512, (3,3), padding='same',activation='relu')(c4_7)
c4_9 = Conv2D(512, (1,1), padding='same',activation='relu')(c4_8)
c4_10 = Conv2D(1024, (3,3), padding='same',activation='relu')(c4_9)
m4 = MaxPool2D((2, 2), strides=(2,2))(c4_10)
c5_1 = Conv2D(512, (1,1), padding='same',activation='relu')(m4)
c5_2 = Conv2D(1024, (3,3), padding='same',activation='relu')(c5_1)
c5_3 = Conv2D(512, (1,1), padding='same',activation='relu')(c5_2)
c5_4 = Conv2D(1024, (3,3), padding='same',activation='relu')(c5_3)
c5_5 = Conv2D(1024, (3,3), padding='same',activation='relu')(c5_4)
c5_6 = Conv2D(1024, (3,3), strides=(2,2), padding='same',activation='relu')(c5_5)
c6_1 = Conv2D(1024, (3,3), padding='same',activation='relu')(c5_6)
c6_2 = Conv2D(1024, (3,3), padding='same',activation='relu')(c6_1)

f = Flatten()(c6_2)
d1 = Dense(4096, activation='relu')(f)
d2 = Dense(7*7*(3+5*2), activation='sigmoid')(d1)
output = Reshape((7,7,(3+5*2)))(d2)

model = Model(input, output)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 448, 448, 3)]     0         
                                                                 
 conv2d_24 (Conv2D)          (None, 224, 224, 64)      9472      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 112, 112, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 112, 112, 192)     110784    
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 56, 56, 192)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 56, 56, 128)       2470

In [14]:
import tensorflow as tf

class Loss():
    def forward(self, pred, target):
        batch_size = tf.shape(pred)[0] 
        N = 2*5+3
        coord_mask = target[:,:,:,4]>0
        noobj_mask = target[:,:,:,4]==0
        coord_mask = tf.expand_dims(coord_mask, axis=4)
        coord_mask = tf.tile(coord_mask, (1,1,1,(2*5+3)))
        noobj_mask = tf.expand_dims(noobj_mask, axis=4)
        noobj_mask = tf.tile(noobj_mask, (1,1,1,(2*5+3)))

        coord_pred = tf.reshape(pred[coord_mask], [-1, N])

        bbox_pred = tf.reshape(coord_pred[:,:10], [-1,5])
        class_pred = coord_pred[:,10:]

        coord_target = tf.reshape(target[coord_mask], [-1, N])

        bbox_target = tf.reshape(coord_target[:,:10], [-1,5])
        class_target = coord_target[:,10:]

        noobj_pred = tf.reshape(pred[noobj_mask], [-1, N])
        noobj_target = tf.reshape(target[noobj_mask], [-1, N])

        noobj_conf_mask = tf.zeros(tf.shape(noobj_pred))
        for b in range(2):
            noobj_conf_mask[:,4+b*5] = 1
        noobj_pred_conf = noobj_pred[noobj_conf_mask]
        noobj_target_conf = noobj_target[noobj_conf_mask]

        loss_noobj = tf.reduce_sum(tf.keras.losses.MSE(noobj_pred_conf, noobj_target_conf))

        coord_response_mask = tf.zeros(tf.shape(bbox_target))
        bbox_target_iou = tf.zeros(tf.shape(bbox_target))

        for i in range(0,tf.shape(bbox_target)[0],2):
            pred=bbox_pred[i:i+2]
            pred_xyxy = tf.zeros(tf.shape(pred))
            pred_xyxy[:,:2] = pred[:,:2]/7.0 - 0.5 * pred[:,2:4]
            pred_xyxy[:,2:4] = pred[:,:2]/7.0 - 0.5 * pred[:,2:4]

            target = bbox_target[i]
            target = tf.reshape(target,[-1,5])
            target_xyxy = tf.zeros(tf.shape(target))
            target_xyxy[:,:2] = target[:,:2]/7.0 - 0.5 * target[:,2:4]
            target_xyxy[:,2:4] = target[:,:2]/7.0 - 0.5 * target[:,2:4]

            iou = self.compute_iou(pred_xyxy[:,:4], target_xyxy[:,:4])
            max_index = tf.argmax(iou)
            max_iou = tf.maximum(iou)

            coord_response_mask[i+max_index] = 1

            bbox_target_iou[i+max_index,4] = max_iou
        
        bbox_pred_response = tf.reshape(bbox_pred[coord_response_mask], [-1,5])
        bbox_target_response = tf.reshape(bbox_target[coord_response_mask], [-1,5])
        target_iou = tf.reshape(bbox_target_iou[coord_response_mask], [-1,5])
        loss_xy = tf.reduce_sum(tf.keras.losses.MSE(bbox_pred_response[:,:2], bbox_target_response[:,:2]))
        loss_wh = tf.reduce_sum(tf.keras.losses.MSE(tf.sqrt(bbox_pred_response[:,2:4]), tf.sqrt(bbox_target_response[:,2:4])))
        loss_obj = tf.reduce_sum(tf.keras.losses.MSE(bbox_pred_response[:,4], target_iou[:,4]))

        loss_class = tf.reduce_sum(tf.keras.losses.MSE(class_pred, class_target))

        loss = loss_xy+loss_wh+loss_obj+loss_noobj+loss_class
        loss = loss/float(batch_size)
        return loss
    
    def compute_single_iou(box1, box2):
        intersect_min = tf.maximum(box1[..., :2], box2[..., :2])
        intersect_max = tf.minimum(box1[..., 2:], box2[..., 2:])
        intersect_wh = tf.maximum(intersect_max - intersect_min, 0.0)
        
        intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
        
        box1_area = (box1[..., 2] - box1[..., 0]) * (box1[..., 3] - box1[..., 1])
        box2_area = (box2[..., 2] - box2[..., 0]) * (box2[..., 3] - box2[..., 1])
        
        union_area = box1_area + box2_area - intersect_area
        
        iou = intersect_area / tf.maximum(union_area, 1e-10)
        
        return iou